In [1]:
import pandas as pd
import os
import numpy as np
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import json


In [2]:
def parse_json_file(json_file):

    resultados = []
    
    if isinstance(json_file, str) and (json_file.endswith('.json') or '/' in json_file):
        with open(json_file, 'r') as file:
            linhas = file.readlines()
    else:
        linhas = json_file.strip().split('\n')
    
    for linha in linhas:
        try:
            obj_externo = json.loads(linha)
            
            resposta_str = obj_externo.get('response', '{}')
            
            resposta_str = resposta_str.strip()
            
            try:
                obj_interno = json.loads(resposta_str)
            except json.JSONDecodeError:
                if resposta_str.startswith('"') and resposta_str.endswith('"'):
                    resposta_str = resposta_str[1:-1]
                obj_interno = json.loads(resposta_str)
            
            sentimento = obj_interno.get('sentiment', '')
            confianca = obj_interno.get('confidence', 0.0)
            
            sentimento = sentimento.lower()
            
            resultados.append({'sentiment': sentimento, 'confidence': confianca})
            
        except Exception as e:
            print(f"Erro ao processar linha: {linha}")
            print(f"Erro: {e}")
    
    df = pd.DataFrame(resultados)
    
    return df

In [3]:
jira_data = pd.read_csv(r'Jira Data')

jira_sentiment_data =  parse_json_file(r"Jira Results")

In [10]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from typing import Dict, Optional

class JiraSentimentAnalyzer:

    COLOR_MAP = {
        'positive': '#2ecc71',
        'neutral': '#95a5a6',
        'negative': '#e74c3c'
    }
    
    def __init__(self, jira_df: pd.DataFrame, sentiment_df: pd.DataFrame):
        self.jira_df = jira_df
        sentiment_df['sentiment'] = sentiment_df['sentiment'].str.lower()
        self.sentiment_df = sentiment_df
        self.merged_data = self._merge_dataframes()
        
    def _merge_dataframes(self) -> pd.DataFrame:
        merged = self.jira_df.copy()
        merged['sentiment'] = self.sentiment_df['sentiment']
        merged['confidence'] = self.sentiment_df['confidence']
        merged['created'] = pd.to_datetime(merged['created'])
        return merged

    def plot_overall_sentiment_distribution(self):
        fig = px.pie(
            self.merged_data,
            names='sentiment',
            title='Overall Project Sentiment Distribution',
            color='sentiment',
            color_discrete_map=self.COLOR_MAP
        )
        fig.update_layout(
            showlegend=True,
            legend_title='Sentiments'
        )
        return fig

    def plot_sentiment_by_issue_type(self):
        sentiment_by_type = pd.crosstab(
            self.merged_data['issuetype'],
            self.merged_data['sentiment']
        )
        
        fig = px.bar(
            sentiment_by_type,
            title='Sentiment Distribution by Issue Type',
            barmode='group',
            color_discrete_map=self.COLOR_MAP
        )
        fig.update_layout(
            xaxis_title='Issue Type',
            yaxis_title='Number of Issues',
            legend_title='Sentiment'
        )
        return fig

    def plot_temporal_sentiment_evolution(self):
        temporal_analysis = (
            self.merged_data
            .groupby([pd.Grouper(key='created', freq='M'), 'sentiment'])
            .size()
            .unstack(fill_value=0)
        )
        
        fig = go.Figure()
        
        for sentiment in temporal_analysis.columns:
            fig.add_trace(
                go.Scatter(
                    x=temporal_analysis.index,
                    y=temporal_analysis[sentiment],
                    name=sentiment,
                    mode='lines+markers',
                    line=dict(color=self.COLOR_MAP[sentiment], width=2),
                    marker=dict(size=6)
                )
            )
        
        fig.update_layout(
            title='Temporal Evolution of Sentiments',
            xaxis_title='Date',
            yaxis_title='Number of Issues',
            hovermode='x unified'
        )
        return fig

    def plot_collaborator_sentiment_evolution(self, collaborator: str):
        collaborator_data = self.merged_data[self.merged_data['assignee'] == collaborator]
        
        temporal_analysis = (
            collaborator_data
            .groupby([pd.Grouper(key='created', freq='M'), 'sentiment'])
            .size()
            .unstack(fill_value=0)
        )
        
        fig = go.Figure()
        
        for sentiment in temporal_analysis.columns:
            fig.add_trace(
                go.Scatter(
                    x=temporal_analysis.index,
                    y=temporal_analysis[sentiment],
                    name=sentiment,
                    mode='lines+markers',
                    line=dict(color=self.COLOR_MAP[sentiment], width=2),
                    marker=dict(size=6)
                )
            )
        
        fig.update_layout(
            title=f'Sentiment Evolution for {collaborator}',
            xaxis_title='Date',
            yaxis_title='Number of Issues',
            hovermode='x unified'
        )
        return fig

    def plot_productivity_evolution(self, collaborator: Optional[str] = None):
        data = self.merged_data if collaborator is None else \
            self.merged_data[self.merged_data['assignee'] == collaborator]
        
        monthly_metrics = (
            data.groupby([pd.Grouper(key='created', freq='ME'), 'assignee'])
            .agg({
                'issue_key': 'count',
                'status': lambda x: (x == 'Concluído').sum(),
            })
            .reset_index()
        )
        
        monthly_metrics['completion_rate'] = monthly_metrics['status'] / monthly_metrics['issue_key']
        
        fig = go.Figure()
        
        for assignee in monthly_metrics['assignee'].unique():
            person_data = monthly_metrics[monthly_metrics['assignee'] == assignee]
            
            fig.add_trace(
                go.Scatter(
                    x=person_data['created'],
                    y=person_data['completion_rate'],
                    name=assignee,
                    mode='lines+markers',
                    hovertemplate=(
                        "<b>%{x}</b><br>" +
                        "Taxa de Conclusão: %{y:.1%}<br>" +
                        "Pessoa: " + assignee
                    )
                )
            )
        
        fig.update_layout(
            height=500,
            width=1000,
            title_text='Evolução da Produtividade por Pessoa',
            xaxis_title='Data',
            yaxis_title='Taxa de Conclusão',
            yaxis_tickformat='.0%',
            hovermode='closest'
        )
        
        return fig

    def plot_workload_completion_relation(self, collaborator: Optional[str] = None):
        data = self.merged_data if collaborator is None else \
            self.merged_data[self.merged_data['assignee'] == collaborator]
        
        monthly_metrics = (
            data.groupby([pd.Grouper(key='created', freq='ME'), 'assignee'])
            .agg({
                'issue_key': 'count',
                'status': lambda x: (x == 'Concluído').sum(),
                'sentiment': lambda x: (x == 'positive').mean()
            })
            .reset_index()
        )
        
        monthly_metrics['completion_rate'] = monthly_metrics['status'] / monthly_metrics['issue_key']
        
        avg_metrics = monthly_metrics.groupby('assignee').agg({
            'issue_key': 'mean',
            'completion_rate': 'mean',
            'sentiment': 'mean'
        }).reset_index()
        
        fig = go.Figure()
        
        fig.add_trace(
            go.Scatter(
                x=avg_metrics['issue_key'],
                y=avg_metrics['completion_rate'],
                mode='markers+text',
                marker=dict(
                    size=15,
                    color=avg_metrics['sentiment'],
                    colorscale='RdYlGn',
                    showscale=True,
                    colorbar=dict(title="% Sentimentos Positivos")
                ),
                text=avg_metrics['assignee'],
                textposition="top center",
                hovertemplate=(
                    "<b>%{text}</b><br>" +
                    "Média de Issues: %{x:.1f}<br>" +
                    "Taxa de Conclusão: %{y:.1%}<br>" +
                    "Sentimentos Positivos: %{marker.color:.1%}"
                )
            )
        )
        
        fig.update_layout(
            height=500,
            width=1000,
            title_text='Relação entre Carga e Taxa de Conclusão',
            xaxis_title='Média de Issues por Mês',
            yaxis_title='Taxa de Conclusão',
            yaxis_tickformat='.0%',
            hovermode='closest'
        )
        
        return fig

    def plot_sentiment_evolution(self, collaborator: Optional[str] = None):
        data = self.merged_data if collaborator is None else \
            self.merged_data[self.merged_data['assignee'] == collaborator]
        
        monthly_metrics = (
            data.groupby([pd.Grouper(key='created', freq='ME'), 'assignee'])
            .agg({
                'sentiment': lambda x: (x == 'positive').mean()
            })
            .reset_index()
        )
        
        fig = go.Figure()
        
        for assignee in monthly_metrics['assignee'].unique():
            person_data = monthly_metrics[monthly_metrics['assignee'] == assignee]
            
            fig.add_trace(
                go.Scatter(
                    x=person_data['created'],
                    y=person_data['sentiment'],
                    name=assignee,
                    mode='lines+markers',
                    hovertemplate=(
                        "<b>%{x}</b><br>" +
                        "Sentimentos Positivos: %{y:.1%}<br>" +
                        "Pessoa: " + assignee
                    )
                )
            )
        
        fig.update_layout(
            height=500,
            width=1000,
            title_text='Evolução dos Sentimentos Positivos',
            xaxis_title='Data',
            yaxis_title='Taxa de Sentimentos Positivos',
            yaxis_tickformat='.0%',
            hovermode='closest'
        )
        
        return fig

    def plot_average_productivity(self, collaborator: Optional[str] = None):
        data = self.merged_data if collaborator is None else \
            self.merged_data[self.merged_data['assignee'] == collaborator]
        
        monthly_metrics = (
            data.groupby([pd.Grouper(key='created', freq='ME'), 'assignee'])
            .agg({
                'issue_key': 'count',
                'status': lambda x: (x == 'Concluído').sum(),
                'sentiment': lambda x: (x == 'positive').mean()
            })
            .reset_index()
        )
        
        monthly_metrics['completion_rate'] = monthly_metrics['status'] / monthly_metrics['issue_key']
        
        avg_metrics = monthly_metrics.groupby('assignee').agg({
            'completion_rate': 'mean',
            'sentiment': 'mean'
        }).reset_index()
        
        fig = go.Figure()
        
        fig.add_trace(
            go.Bar(
                x=avg_metrics['assignee'],
                y=avg_metrics['completion_rate'],
                marker=dict(
                    color=avg_metrics['sentiment'],
                    colorscale='RdYlGn',
                    showscale=True,
                    colorbar=dict(title="% Sentimentos Positivos")
                ),
                hovertemplate=(
                    "<b>%{x}</b><br>" +
                    "Taxa Média de Conclusão: %{y:.1%}<br>" +
                    "Média de Sentimentos Positivos: %{marker.color:.1%}"
                )
            )
        )
        
        fig.update_layout(
            height=500,
            width=1000,
            title_text='Média de Produtividade por Pessoa',
            xaxis_title='Pessoa',
            xaxis_tickangle=45,
            yaxis_title='Taxa Média de Conclusão',
            yaxis_tickformat='.0%',
            hovermode='closest'
        )
        
        return fig

    def plot_team_productivity_timeline(self):
        team_metrics = (
            self.merged_data
            .groupby(pd.Grouper(key='created', freq='M'))
            .agg({
                'issue_key': 'count', 
                'status': lambda x: (x == 'Concluído').sum() 
            })
            .reset_index()
        )
        
        team_metrics['completion_rate'] = team_metrics['status'] / team_metrics['issue_key']
        
        fig = make_subplots(specs=[[{"secondary_y": True}]])
        
        fig.add_trace(
            go.Scatter(
                x=team_metrics['created'],
                y=team_metrics['issue_key'],
                name='Total Issues',
                mode='lines+markers'
            ),
            secondary_y=False
        )
        
        fig.add_trace(
            go.Scatter(
                x=team_metrics['created'],
                y=team_metrics['completion_rate'],
                name='Completion Rate',
                mode='lines+markers'
            ),
            secondary_y=True
        )
        
        fig.update_layout(
            title='Team Productivity Timeline',
            xaxis_title='Date',
            hovermode='x unified'
        )
        
        fig.update_yaxes(title_text="Number of Issues", secondary_y=False)
        fig.update_yaxes(
            title_text="Completion Rate", 
            secondary_y=True,
            tickformat='.0%'
        )
        
        return fig

    def plot_workload_analysis(self):
        workload_data = (
            self.merged_data
            .groupby(['assignee', pd.Grouper(key='created', freq='ME')])  
            .agg({
                'issue_key': 'count',  
                'sentiment': lambda x: (x == 'negative').mean(), 
                'status': lambda x: (x == 'Concluído').mean() 
            })
            .reset_index()
        )
        
        fig = make_subplots(
            rows=2, 
            cols=1,
            subplot_titles=(
                'Issues Atribuídas por Pessoa ao Longo do Tempo',
                'Indicadores de Sobrecarga por Pessoa'
            ),
            vertical_spacing=0.22
        )
        
        for assignee in workload_data['assignee'].unique():
            person_data = workload_data[workload_data['assignee'] == assignee]
            
            fig.add_trace(
                go.Scatter(
                    x=person_data['created'],
                    y=person_data['issue_key'],
                    name=assignee,
                    mode='lines+markers',
                    hovertemplate=(
                        "<b>%{x}</b><br>" +
                        "Issues: %{y}<br>" +
                        "Pessoa: " + assignee
                    )
                ),
                row=1, col=1
            )
        
        summary_data = workload_data.groupby('assignee').agg({
            'issue_key': 'mean',  
            'sentiment': 'mean',  
            'status': 'mean'  
        }).reset_index()
        
        summary_data['issue_key_norm'] = summary_data['issue_key'] / summary_data['issue_key'].max()
        summary_data['workload_index'] = (
            (summary_data['issue_key_norm'] * 0.4) + 
            (summary_data['sentiment'] * 0.3) + 
            ((1 - summary_data['status']) * 0.3)  
        )
        
        colors = px.colors.diverging.RdYlGn_r
        normalized_indices = (summary_data['workload_index'] - summary_data['workload_index'].min()) / \
                            (summary_data['workload_index'].max() - summary_data['workload_index'].min())
        bar_colors = [colors[int(i * (len(colors)-1))] for i in normalized_indices]
        
        fig.add_trace(
            go.Bar(
                x=summary_data['assignee'],
                y=summary_data['workload_index'],
                marker_color=bar_colors,
                hovertemplate=(
                    "<b>%{x}</b><br>" +
                    "Índice de Sobrecarga: %{y:.2f}<br>" +
                    "Média de Issues/Mês: %{customdata[0]:.1f}<br>" +
                    "Taxa de Sentimentos Negativos: %{customdata[1]:.1%}<br>" +
                    "Taxa de Conclusão: %{customdata[2]:.1%}"
                ),
                customdata=np.column_stack((
                    summary_data['issue_key'],
                    summary_data['sentiment'],
                    summary_data['status']
                ))
            ),
            row=2, col=1
        )
        
        fig.update_layout(
            height=800,
            title_text='Análise de Sobrecarga de Trabalho da Equipe',
            showlegend=True,
            hovermode='closest'
        )
        
        fig.update_xaxes(title_text='Data', row=1, col=1)
        fig.update_xaxes(title_text='Pessoa', row=2, col=1)
        fig.update_yaxes(title_text='Número de Issues', row=1, col=1)
        fig.update_yaxes(title_text='Índice de Sobrecarga', row=2, col=1)
        
        return fig